# Measuring Acoustic Voice Quality Index (AVQI)


The Parselmouth `voice_analysis.ipynb` (link?) example illustrated how you can use Parselmouth to reproduce Praat's Voice Report in Python. Praat actually can compute other voice related measures: perhaps the most important one, the cepstral peak prominence. This useful feature is somewhat buried in Praat's GUI, but you can compute it easily using Parselmouth.

In this example, we follow the Praat script put together by Youri Maryn and David Weenink for their publication:

> Y. Maryn and D. Weenink, “Objective dysphonia measures in the program Praat: Smoothed cepstral peak prominence and acoustic voice quality index,Journal of Voice, vol. 29, no. 1, pp. 35–43, Jan. 2015, doi: 10.1016/j.jvoice.2014.06.015.

And compute cepstral peak prominence and acoustic voice quality index measures of our sample acoustic data.

## Step 0. Get necessary packages

We start out by importing parselmouth and other Python packages

In [ ]:
import parselmouth

import numpy as np
import matplotlib.pyplot as plt

## Step 1. Read the acoustic data and remove unvoiced samples

Fist thing to do is to open and read in the audio file and then only use the voiced samples.

In [ ]:
# Load acoustic data from the WAV file
wavfile = "docs/examples/audio/the_north_wind_and_the_sun.wav"
cv = parselmouth.Sound(wavfile)

# plot the original
plt.figure()
plt.plot(cv.xs(), cv.values.T)
plt.axvline(cv.xmin,ls=":",c="tab:red")
plt.axvline(cv.xmax,ls=":",c="tab:red")
plt.xlim((cv.xmin, cv.xmax))
plt.xlabel("time [s]")
plt.ylabel("amplitude")
plt.title("Original waveform")


def truncate_unvoiced(cv):

    # analyze pitch with Praat
    pitch = cv.to_pitch_cc(None,75,15,False, 0.03, 0.45, 0.01, 0.35, 0.14, 600) # To Pitch (cc)... 0 75 15 no 0.03 0.45 0.01 0.35 0.14 600
    f0 = pitch.selected_array['frequency'] # get the f0 estimates as an array
    fs = 1/cv.dt # sample rate

    # remove unvoiced samples (if unvoiced, f0 is 0)
    voiced = f0>0.0 # True if frame is voiced
    N = len(voiced)
    mask = np.ones(len(cv),dtype=bool) # true to keep the corresponding sound sample
    dt = pitch.dt # pitch frame time offset
    i0 = next((j for j in range(N) if not voiced[j]),N) if voiced[0] else 0
    while i0<N: # i0 initialized to the beginning of the first unvoiced segment 
        n0 = int(i0*dt*fs)

        # find the end of unvoiced segment
        i = next((j for j in range(i0+1,N) if voiced[j]),N)
        if i<N:
            n = int(i*dt*fs)
            mask[n0:n] = False
            i0 = next((j for j in range(i+1,N) if not voiced[j]),N)
        else: # last segment
            mask[n0:] = False
            i0 = i

    # return the new sound object
    return parselmouth.Sound(cv.values[:,mask],fs)

sound = truncate_unvoiced(cv)
fs = 1/sound.dt # sample rate

# Plot the waveform and analysis window
plt.figure()
plt.plot(sound.xs(), sound.values.T)
plt.axvline(sound.xmin,ls=":",c="tab:red")
plt.axvline(sound.xmax,ls=":",c="tab:red")
plt.xlim((sound.xmin, sound.xmax))
plt.xlabel("time [s]")
plt.ylabel("amplitude")
plt.title("waveform with unvoiced samples removed")
plt.show()


In [ ]:
spectrogram = sound.to_spectrogram(0.03, 4000, 0.002, 20, "GAUSSIAN") # To Spectrogram... 0.03 4000 0.002 20 Gaussian
pitch = sound.to_pitch_cc(None,75,15,False,0.03,0.45,0.01,0.35,0.14) # To Pitch (cc)... 0 75 15 no 0.03 0.45 0.01 0.35 0.14 600
pulses = sound.to_point_process_periodic(50,400) # To PointProcess (periodic, cc)... 50 400"
ltas = sound.to_ltas(100) # To Ltas... 1 <- typo by Maryn???
powercepstrogram = sound.to_power_cepstrogram(60,0.002,5000,50) # To PowerCepstrogram... 60 0.002 5000 50
powercepstrum = powercepstrogram.to_power_cepstrum(0.1) # To PowerCepstrum (slice)... 0.1


In [ ]:
plt.figure()
plt.pcolormesh(spectrogram.x_grid(), spectrogram.y_grid(), 10 * np.log10(spectrogram.values), cmap='afmhot')
plt.ylim([spectrogram.ymin, spectrogram.ymax])
plt.xlabel("time [s]")
plt.ylabel("frequency [Hz]")

plt.figure()
plt.plot(ltas.xs(),ltas.values.T)
plt.xlim((0,len(ltas)*ltas.get_bin_width()))
plt.xlabel("Frequency (Hz)")
plt.ylabel("Sound pressure level (dB/Hz)")

plt.figure()
plt.pcolormesh(powercepstrogram.x_grid(), powercepstrogram.y_grid(), 10 * np.log10(powercepstrogram.values), cmap='afmhot')
plt.ylim((0.00303, 0.01667))
plt.ylabel("Quefrency (s)")
plt.xlabel("Time (s)")

# select PowerCepstrum avqi_0_100
plt.figure()
plt.plot(powercepstrum.xs(),10*np.log10(powercepstrum.values.T))
plt.xlim((0,powercepstrum.xmax))
plt.xlabel("Quefrency (s)")
plt.ylabel("Amplitude (dB)")
plt.show()


In [ ]:
cpps = powercepstrogram.get_cpps(False,0.01,0.001,60,330,0.05,"PARABOLIC",0.001,0,"STRAIGHT","ROBUST") # Get CPPS... no 0.01 0.001 60 330 0.05 
print(f"Smoothed cepstral peak prominence (CPPS)={cpps:.2f}")

hnr = pitch.get_mean_strength('hnr_db', 0,0)
print(f"Harmonics-to-noise ratio={hnr:.2f} dB")

percentShimmer = pulses.get_shimmer_local(sound,0,0,0.0001,0.02,1.3,1.6) # Get shimmer (local)... 0 0 0.0001 0.02 1.3 1.6
shim = percentShimmer*100
shdb = pulses.get_shimmer_local_db(sound,0, 0, 0.0001, 0.02, 1.3, 1.6) # Get shimmer (local_dB)... 0 0 0.0001 0.02 1.3 1.6
print(f"Shimmer local={shim:.2f}%")
print(f"Shimmer local dB={shdb:.2f} dB")

slope = ltas.get_slope(0,1000,1000,10000,"ENERGY") # Get slope... 0 1000 1000 10000 energy
ltas.compute_trend_line(1,10000) # Compute trend line... 1 10000
tilt = ltas.get_slope(0,1000,1000,10000,"ENERGY") # Get slope... 0 1000 1000 10000 energy

print(f"Slope of LTAS={slope:.2f} dB")
print(f"Tilt of trendline through LTAS={tilt:.2f} dB")

avqi = 9.072-0.245*cpps-0.161*hnr-0.470*shim+6.158*shdb-0.071*slope+0.170*tilt
print(f"AVQI={avqi:.2f}")
